<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/2%20-%20Overfitting%20e%20Dropout/testing_e_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing del Modello
Per verificare la reale capacità predittiva di una Rete Neurale (ma anche di un qualsiasi altro modello di Machine Learning) dobbiamo testarlo su dati che non ha già visto durante l'addestramento.

## Dipendenze

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Prepariamo il Dataset

In [0]:
csv_url = "https://raw.githubusercontent.com/ProfAI/tf00/master/datasets/math_class/math_class_deep.csv"
df = pd.read_csv(csv_url)

to_encode = ["sex", "paid", "higher", "internet", "romantic"]
df[to_encode] = pd.get_dummies(df[to_encode], drop_first=True)

X = df.drop(["student_id","promoted"], axis=1).values
y = df["promoted"].values

Tramite la funzione *train_test_split* di scikit-learn dividiamo il dataset in due subset:
* **Train Set** set di dati che utilizzeremo per addestrare il modello, contenenente il 70% degli esempi del dataset.
* **Test Set** set di dati che utilizzeremo per testare il modello, contenenente il restante 30% degli esempi del dataset.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Underfitting
Addestriamo un primo semplice modello di regressione logistica, tramite il metodo *summary* possiamo vedere il numero totale di parametri che lo compongono.

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"),
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 13        
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


Sono appena 13. Compiliamo il modello e avviamo l'addestramento sul train set per 100 epoche.

In [5]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
9/9 [==============================] - 0s 4ms/step - loss: 1.0760 - accuracy: 0.6703
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 1.0086 - accuracy: 0.6703
Epoch 3/100
9/9 [==============================] - 0s 3ms/step - loss: 0.9486 - accuracy: 0.6703
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8989 - accuracy: 0.6703
Epoch 5/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8580 - accuracy: 0.6594
Epoch 6/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8331 - accuracy: 0.6558
Epoch 7/100
9/9 [==============================] - 0s 4ms/step - loss: 0.8150 - accuracy: 0.6413
Epoch 8/100
9/9 [==============================] - 0s 3ms/step - loss: 0.8014 - accuracy: 0.6268
Epoch 9/100
9/9 [==============================] - 0s 3ms/step - loss: 0.7951 - accuracy: 0.6159
Epoch 10/100
9/9 [==============================] - 0s 3ms/step - loss: 0.7907 - accuracy: 0.6123
Epoch 11/100
9/9 [===========

Al termine dell'addestramento le metriche, loss e accuracy, sul set di addestramento sono piuttosto scarse, verifichiamole anche sul set di test, possiamo farlo utilizzando il metodo *evaluate*.

In [6]:
metrics_train = model.evaluate(X_train, y_train)
metrics_test = model.evaluate(X_test, y_test)
print("Accuracy sul set di addestramento: %.2f - Loss sul set di addestramento %.4f" % (metrics_train[1], metrics_train[0]))
print("Accuracy sul set di test: %.2f - Loss sul set di test %.4f" % (metrics_test[1], metrics_test[0]))

4/4 [==============================] - 0s 3ms/step - loss: 0.6173 - accuracy: 0.6807
Accuracy sul set di addestramento: 0.68 - Loss sul set di addestramento 0.6275
Accuracy sul set di test: 0.68 - Loss sul set di test 0.6173


Le metriche su Train Set e Test Set sono molto simili e in entrambi sono troppo scarse per poter considerare il nostro modello valido. In questi casi si parla di **Underfitting**, il nostro modello non è riuscito ad estrarre sufficienti informazioni dal Train Set per poter apprendere le regole necessarie ad eseguire una predizione.

## Overfitting
Addestriamo ora una Rete Neurale Profonda.

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"),
    keras.layers.Dense(128, input_shape=[X.shape[1]], activation="relu"),
    keras.layers.Dense(1, input_shape=[X.shape[1]], activation="sigmoid"),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               1664      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 18,305
Trainable params: 18,305
Non-trainable params: 0
_________________________________________________________________


Questa volta i parametri sono ben 18.305, oltre 1000x più del modello precedente. Compiliamo il modello e avviamo di nuovo l'addestramento sul Train Set per 500 epoche.

In [8]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=500)

Epoch 1/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6878 - accuracy: 0.6703
Epoch 2/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6424 - accuracy: 0.6413
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6242 - accuracy: 0.6739
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 0.6040 - accuracy: 0.7174
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5961 - accuracy: 0.6848
Epoch 6/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5898 - accuracy: 0.7029
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5871 - accuracy: 0.7101
Epoch 8/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5805 - accuracy: 0.7319
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5832 - accuracy: 0.7065
Epoch 10/500
9/9 [==============================] - 0s 3ms/step - loss: 0.5798 - accuracy: 0.7428
Epoch 11/500
9/9 [===========

Ecco che ora le metriche sono di tutto rispetto, ma questo non è sufficiente a poter considerare il nostro modello funzionante, infatti dobbiamo verificare che sia in grado di riconoscere esempi che non ha visto durante l'addestramento, per farlo possiamo utilizzare il set di test.

In [0]:
metrics_train = model.evaluate(X_train, y_train)
metrics_test = model.evaluate(X_test, y_test)
print("Accuracy sul set di addestramento: %.2f - Loss sul set di addestramento %.4f" % (metrics_train[1], metrics_train[0]))
print("Accuracy sul set di test: %.2f - Loss sul set di test %.4f" % (metrics_test[1], metrics_test[0]))

4/4 [==============================] - 0s 2ms/step - loss: 2.2079 - accuracy: 0.6218
Accuracy sul set di addestramento: 0.99 - Loss sul set di addestramento 0.0428
Accuracy sul set di test: 0.62 - Loss sul set di test 2.2079


Nonostante le metriche sul Train Set siano molto buone, non si può dire altrettanto per quelle Test Set, quello che è accaduto è che il modello ha *memorizzato* gli esempi del set di addestramento, piuttosto che usarle per apprendere. Questa condizione è conosciuta come **Overfitting**

## Per Ricapitolare
* **Underfitting**: il modello non è riuscto ad apprendere a sufficienza, sia le metriche sul Train Set che sul Test Set sono scarse.
* **Overfitting**: il modello ha memorizzato i dati del Train Set e non riesce a generalizzare su nuovi dati, le metriche sul Train Set sono eccessivamente buone, mentre quelle sul Test Set sono più scarse.